In [1]:
import cv2
import numpy as np
import json
from datetime import timedelta

In [2]:
def detectar_logo_no_video(video_path, logo_path, min_match_count=10, ratio_test=0.7):
    # Carregar a logotipo target em escala de cinza
    logo_image = cv2.imread(logo_path, cv2.IMREAD_GRAYSCALE)
    if logo_image is None:
        raise IOError("Não foi possível carregar a imagem da logotipo.")

    # Iniciar a captura de vídeo
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError("Não foi possível abrir o vídeo.")

    # Inicializar o detector SIFT
    sift = cv2.SIFT_create()

    # Detectar keypoints e descritores na logotipo
    kp_logo, des_logo = sift.detectAndCompute(logo_image, None)
    if des_logo is None:
        raise ValueError("Não foi possível encontrar descritores na logotipo.")

    # Configurar o FLANN Matcher
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)  # Número de verificações
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # Parâmetros de detecção
    logo_present = False
    start_time = None
    end_time = None
    detections = []

    # Obter frames por segundo (FPS) do vídeo
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0:
        raise ValueError("FPS do vídeo é 0, verifique o arquivo de vídeo.")

    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # Fim do vídeo

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detectar keypoints e descritores no frame
        kp_frame, des_frame = sift.detectAndCompute(frame_gray, None)

        if des_frame is not None:
            # Encontrar correspondências usando FLANN
            try:
                matches = flann.knnMatch(des_logo, des_frame, k=2)
            except cv2.error as e:
                print(f"Erro durante a correspondência de características: {e}")
                matches = []

            # Aplicar o ratio test de Lowe
            good_matches = []
            for m_n in matches:
                if len(m_n) != 2:
                    continue
                m, n = m_n
                if m.distance < ratio_test * n.distance:
                    good_matches.append(m)

            if len(good_matches) > min_match_count:
                # Encontrou a logotipo no frame
                if not logo_present:
                    logo_present = True
                    start_time = frame_count / fps
            else:
                if logo_present:
                    logo_present = False
                    end_time = frame_count / fps
                    detections.append({
                        "start": str(timedelta(seconds=start_time)),
                        "end": str(timedelta(seconds=end_time))
                    })
        else:
            if logo_present:
                logo_present = False
                end_time = frame_count / fps
                detections.append({
                    "start": str(timedelta(seconds=start_time)),
                    "end": str(timedelta(seconds=end_time))
                })

        frame_count += 1

    # Se o vídeo terminar com a logotipo presente
    if logo_present:
        end_time = frame_count / fps
        detections.append({
            "start": str(timedelta(seconds=start_time)),
            "end": str(timedelta(seconds=end_time))
        })

    cap.release()

    # Estruturar o resultado
    output = {"detections": detections}

    # Salvar em um arquivo JSON
    #with open(output_json_path, 'w') as json_file:
        #json.dump(output, json_file, indent=4, ensure_ascii=False)

    #print(output)

    return output

In [3]:
logo_path = "../target/logo_target.png"
video_path = '../videos/15s-canto-direito.mp4'

In [4]:
detectar_logo_no_video(video_path, logo_path)

{'detections': [{'start': '0:00:04.733333', 'end': '0:00:04.800000'},
  {'start': '0:00:05.500000', 'end': '0:00:05.533333'},
  {'start': '0:00:05.833333', 'end': '0:00:06.466667'},
  {'start': '0:00:06.500000', 'end': '0:00:07.433333'},
  {'start': '0:00:07.666667', 'end': '0:00:07.700000'},
  {'start': '0:00:07.866667', 'end': '0:00:07.900000'},
  {'start': '0:00:07.933333', 'end': '0:00:08'}]}

In [3]:
status = False

if not status:
    print('entrei')
else:
    print('não entrei')

entrei
